In [1]:
from pyscbwrapper import SCB
import pandas as pd

In [2]:
scb = SCB('sv', 'BE', 'BE0101', 'BE0101A', 'BefolkManad')

In [3]:
scb.get_variables()['tabellinnehåll']

['Folkmängden per månad']

In [4]:
regions = [region for region in scb.get_variables()['region'] if region == 'Riket' or region[-3:] == 'län']

In [5]:
vars_ = scb.get_variables()
scb.set_query(region=regions, 
              ålder=vars_['ålder'],
              kön=vars_['kön'],
              månad=vars_['månad'][-1],)

In [6]:
data = scb.get_data()['data']

In [7]:
df = pd.DataFrame(data)
df['region'] = [k[0] for k in df.key]
df['age'] = [k[1] for k in df.key]
df['sex'] = [k[2] for k in df.key]
df['time'] = [k[3] for k in df.key]
df['value'] = [int(k[0]) for k in df['values']]
df = df[df.columns[2:]]
df

,region,age,sex,time,value
0,00,0,1,2020M02,9623
1,00,0,2,2020M02,8973
2,00,1,1,2020M02,59624
3,00,1,2,2020M02,56084
4,00,2,1,2020M02,61080
...,...,...,...,...,...
4439,25,98,2,2020M02,51
4440,25,99,1,2020M02,6
4441,25,99,2,2020M02,31
4442,25,100+,1,2020M02,11


In [8]:
region_mappings = {k: v for k, v in 
                   zip(scb.info()['variables'][0]['values'], 
                       scb.info()['variables'][0]['valueTexts'])}
df['region'] = [region_mappings[reg] for reg in df['region']]

scb_sex = scb.get_variables()['kön']
df['sex'] = [scb_sex[int(v)-1] for v in df['sex']]

In [9]:
# Bin age data
df.loc[df.age == '100+','age'] = 100
df['age'] = df['age'].astype(int)

lbls = [f'{v}0-{(v+1)*10-1}' for v in range(10)]
lbls[0] = '0-9'
lbls[-1] = '90+'
df['agegroup'] = pd.cut(df['age'], 10, precision=3, labels=lbls)

In [12]:
df.to_pickle('pop_2020-02.pkl')